In [1]:
!pip -q install scapy

from scapy.all import IP, TCP, UDP, ICMP, Raw, wrpcap
import pandas as pd, numpy as np, re, os, random


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 16.4 MB/s eta 0:00:00


In [3]:
# 1) Mount Drive
from google.colab import drive
drive.mount('/content/drive')

# 2) Search Drive for the TON_IoT network CSV (handles different names/locations)
import glob, os, textwrap

root = "/content/drive/MyDrive"
candidates = []
patterns = [
    "**/TON_IoT/**/*.csv",
    "**/TON_IoT/**/*network*.csv",
    "**/Train_Test_Network*.csv",
    "**/train_test_network*.csv",
]
for pat in patterns:
    candidates += glob.glob(os.path.join(root, pat), recursive=True)

# Keep only plausible files
candidates = [p for p in candidates if os.path.isfile(p)]
candidates = sorted(set(candidates))

print(f"Found {len(candidates)} CSV file(s). Top matches:")
for p in candidates[:10]:
    print(" -", p)

# If you see the right file printed, pick it; otherwise tweak patterns above.
# Try to auto-pick a good one:
csv_path = None
for p in candidates:
    name = os.path.basename(p).lower()
    if "network" in name or "train_test" in name:
        csv_path = p
        break

# Fallback to first candidate
if csv_path is None and candidates:
    csv_path = candidates[0]

assert csv_path is not None, "Could not find the TON_IoT network CSV in Drive. Scroll the printed list and copy the right path."

print("\nUSING CSV:", csv_path)


Mounted at /content/drive
Found 51 CSV file(s). Top matches:
 - /content/drive/MyDrive/MedSecureAI - Cybersecurity Team/MOP_UseCase11_Project/datasets/TON_IoT/Processed_datasets/Processed_IoT_dataset/IoT_Fridge.csv
 - /content/drive/MyDrive/MedSecureAI - Cybersecurity Team/MOP_UseCase11_Project/datasets/TON_IoT/Processed_datasets/Processed_IoT_dataset/IoT_GPS_Tracker.csv
 - /content/drive/MyDrive/MedSecureAI - Cybersecurity Team/MOP_UseCase11_Project/datasets/TON_IoT/Processed_datasets/Processed_IoT_dataset/IoT_Garage_Door.csv
 - /content/drive/MyDrive/MedSecureAI - Cybersecurity Team/MOP_UseCase11_Project/datasets/TON_IoT/Processed_datasets/Processed_IoT_dataset/IoT_Modbus.csv
 - /content/drive/MyDrive/MedSecureAI - Cybersecurity Team/MOP_UseCase11_Project/datasets/TON_IoT/Processed_datasets/Processed_IoT_dataset/IoT_Motion_Light.csv
 - /content/drive/MyDrive/MedSecureAI - Cybersecurity Team/MOP_UseCase11_Project/datasets/TON_IoT/Processed_datasets/Processed_IoT_dataset/IoT_Thermostat

In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv(csv_path)

# Normalise likely column names
rename_map = {
    'Src_IP':'src_ip', 'srcIP':'src_ip', 'source_ip':'src_ip',
    'Dst_IP':'dst_ip', 'dstIP':'dst_ip', 'destination_ip':'dst_ip',
    'Src_Port':'src_port', 'Source_Port':'src_port',
    'Dst_Port':'dst_port', 'Destination_Port':'dst_port',
    'Protocol':'proto', 'protocol':'proto',
    'Label':'label', 'class':'label'
}
df = df.rename(columns={k:v for k,v in rename_map.items() if k in df.columns})

# Ensure needed columns exist
needed = ['src_ip','dst_ip','src_port','dst_port','proto','service','src_bytes','dst_bytes','label']
for c in needed:
    if c not in df.columns: df[c] = np.nan

# Take a manageable sample for first PCAP
df = df.sample(n=min(len(df), 3000), random_state=42).reset_index(drop=True)
df.head()


/tmp/ipython-input-828409183.py:4: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


,ts,src_ip,src_port,dst_ip,dst_port,proto,service,duration,src_bytes,dst_bytes,...,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,type
0,1556025777,192.168.1.32,50266,192.168.1.186,27228,tcp,-,0.0,0,0,...,0,0,-,-,-,-,-,-,1,scanning
1,1554273808,127.0.0.1,42100,127.0.0.1,7878,tcp,-,0.0,0,0,...,0,0,-,-,-,-,-,-,0,normal
2,1556025649,192.168.1.152,34422,192.168.1.32,10644,tcp,-,0.0,0,0,...,0,0,-,-,-,-,-,-,1,scanning
3,1556025616,192.168.1.30,22069,192.168.1.180,33883,tcp,-,0.0,0,0,...,0,0,-,-,-,-,-,-,1,scanning
4,1554258881,192.168.1.79,39062,192.168.1.255,15600,udp,-,0.0,0,0,...,0,0,-,-,-,-,-,-,0,normal


In [5]:
!pip -q install scapy
from scapy.all import IP, TCP, UDP, ICMP, Raw, wrpcap
import re, random, os

ip_pat = re.compile(r'^(?:\d{1,3}\.){3}\d{1,3}$')
def is_ip(x):
    try:
        return bool(ip_pat.match(str(x))) and all(0 <= int(o) <= 255 for o in str(x).split('.'))
    except: return False

private_ranges = [('10.0.0.0','10.255.255.255'),('172.16.0.0','172.31.255.255'),('192.168.0.0','192.168.255.255')]
def ip2int(ip): a,b,c,d = map(int, ip.split('.')); return (a<<24)+(b<<16)+(c<<8)+d
def int2ip(x): return ".".join(map(str,[(x>>24)&255,(x>>16)&255,(x>>8)&255,x&255]))
def rand_priv_ip():
    lo,hi = random.choice([(ip2int(a),ip2int(b)) for a,b in private_ranges])
    return int2ip(random.randint(lo,hi))

service_to_port = {'http':80,'https':443,'ftp':21,'smtp':25,'telnet':23,'ssh':22,'domain':53,'pop3':110,'imap':143,'ntp':123,'dns':53,'mqtt':1883}
def norm_service(s):
    if pd.isna(s): return ''
    return str(s).lower().strip().replace(' ', '')

def choose_ports(row):
    try: sport = int(row.get('src_port', np.nan))
    except: sport = np.nan
    try: dport = int(row.get('dst_port', np.nan))
    except: dport = np.nan
    if not (isinstance(sport,int) and 1<=sport<=65535): sport = random.randint(1024,65535)
    if not (isinstance(dport,int) and 1<=dport<=65535):
        dport = service_to_port.get(norm_service(row.get('service','')), random.randint(1024,65535))
    return sport, dport

def norm_proto(p):
    p = str(p).lower().strip()
    if p in ['6','tcp']: return 'tcp'
    if p in ['17','udp']: return 'udp'
    if p in ['1','icmp']: return 'icmp'
    return 'tcp'

def payload_from_bytes(src_b, dst_b, cap=600):
    def safe_int(v):
        try: return int(v)
        except: return 0
    L = max(1, min(max(safe_int(src_b), safe_int(dst_b), 1), cap))
    return bytes([0x58])*L

def row_to_packet(row):
    sip = row.get('src_ip'); dip = row.get('dst_ip')
    if not is_ip(sip): sip = rand_priv_ip()
    if not is_ip(dip) or dip == sip:
        dip = rand_priv_ip()
        while dip == sip: dip = rand_priv_ip()
    proto = norm_proto(row.get('proto','tcp'))
    sport, dport = choose_ports(row)
    payload = payload_from_bytes(row.get('src_bytes',0), row.get('dst_bytes',0))
    if proto == 'udp':   pkt = IP(src=sip,dst=dip)/UDP(sport=sport,dport=dport)/Raw(load=payload)
    elif proto == 'icmp': pkt = IP(src=sip,dst=dip)/ICMP(type=8)/Raw(load=payload[:128])
    else:                pkt = IP(src=sip,dst=dip)/TCP(sport=sport,dport=dport,flags='PA')/Raw(load=payload)
    return pkt

# Build and write PCAPs
packets = [row_to_packet(r) for _, r in df.iterrows()]
out_dir = "/content/drive/MyDrive/MedSecureAI/pcaps"
os.makedirs(out_dir, exist_ok=True)
small_pcap = os.path.join(out_dir, "toniotsynth_small.pcap")
wrpcap(small_pcap, packets)
small_pcap


'/content/drive/MyDrive/MedSecureAI/pcaps/toniotsynth_small.pcap'